In [1]:
import spacy
nlp = spacy.load('pl_spacy_model_morfeusz')
# pl_spacy_model_morfeusz now requires the following dependencies:
# - morfeusz2
# - distance
# - keras
# - numpy

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [2]:
# Print the names of the pipeline components
print(nlp.pipe_names)

# Print the full pipeline of (name, component) tuples
print(nlp.pipeline)
# The Preprocessor is not listed as it is a substitute for the tokenizer
# The pipeline does not include the tagger, as it is contained within the Preprocessor

['parser', 'ner', 'flexer']
[('parser', <spacy.pipeline.pipes.DependencyParser object at 0x000002102DB54EE8>), ('ner', <spacy.pipeline.pipes.EntityRecognizer object at 0x000002102DB54F48>), ('flexer', <pl_spacy_model_morfeusz.flexer.Flexer object at 0x0000021033F61240>)]


In [3]:
# List the tokens and UPOS tags
doc = nlp("Granice mojego języka oznaczają granice mojego świata") # ~Wittgenstein
for token in doc:
    print(token.text, token.lemma_, token.pos_)

Granice granica NOUN
mojego mój ADJ
języka język NOUN
oznaczają oznaczać VERB
granice granica NOUN
mojego mój ADJ
świata świat NOUN


In [4]:
# In pl_spacy_model_morfeusz token.tag_ stores the language-specific POS tag (XPOS), while token._.feats stores an the morphosyntactic features (e.g. case or grammatical gender)

# The extended tag is the concatenation of the  XPOS tag, and all the morphosyntactic features
# to get the extended tag you have to join the both together

def extended_tag(token):
  if token._.feats == "":
    return token.tag_
  else:
    return ":".join([token.tag_, token._.feats])

for token in doc:
    print('\t'.join([token.orth_, token.tag_, token._.feats, extended_tag(token)]))

Granice	subst	pl:nom:f	subst:pl:nom:f
mojego	adj	sg:gen:m3:pos	adj:sg:gen:m3:pos
języka	subst	sg:gen:m3	subst:sg:gen:m3
oznaczają	fin	pl:ter:imperf	fin:pl:ter:imperf
granice	subst	pl:acc:f	subst:pl:acc:f
mojego	adj	sg:gen:m3:pos	adj:sg:gen:m3:pos
świata	subst	sg:gen:m3	subst:sg:gen:m3


In [5]:
# Tokenization, Tagging, Lemmatization and Dependency Parsing in a pretty table
import pandas as pd

attribs = ['orth_', 'lemma_', 'tag_', 'pos_', 'dep_', 'head']
table = [{att : tok.__getattribute__(att) for att in attribs} for tok in doc]
df = pd.DataFrame(table)
print(df[attribs])

       orth_    lemma_   tag_  pos_      dep_       head
0    Granice   granica  subst  NOUN     nsubj  oznaczają
1     mojego       mój    adj   ADJ  det:poss     języka
2     języka     język  subst  NOUN  nmod:arg    Granice
3  oznaczają  oznaczać    fin  VERB      ROOT  oznaczają
4    granice   granica  subst  NOUN       obj  oznaczają
5     mojego       mój    adj   ADJ  det:poss     świata
6     świata     świat  subst  NOUN  nmod:arg    granice


In [6]:
# Play with the pos values
for token in doc:
    # Check if the current token is a proper noun
    if token.pos_ == "ADJ":
        # Check if the next token is a verb
        if doc[token.i + 1].pos_ == "NOUN":
            print("Found adjective before a noun:", token.text)

Found adjective before a noun: mojego
Found adjective before a noun: mojego


In [7]:
# DEP visualization
from spacy import displacy
displacy.render(doc, style="dep")

In [8]:
# Parsing multiple sentences
doc = nlp("Prof. Nowak wyszedł z domu spóźniony. Potknął się o próg i upadł. \"Nic się nie stało?\" - pytała sąsiadka.")
for sent in doc.sents:
    print(sent)

Prof. Nowak wyszedł z domu spóźniony.
Potknął się o próg i upadł.
"Nic się nie stało?" - pytała sąsiadka.


In [9]:
displacy.render(doc, style = 'dep', options = {"compact": True})

In [10]:
# Named entities
doc = nlp("Selekcjoner reprezentacji Polski Jerzy Brzęczek \
        powołał piłkarzy na mecze eliminacji mistrzostw Europy 2020,\
        6 września Polacy zagrają w Lublanie ze Słowenią.") # ~Rzeczpospolita 

for entity in doc.ents:
    print(entity.text, entity.label_)

Polski placeName
Jerzy Brzęczek persName
Europy geogName
2020 date
września date
Polacy placeName
Lublanie placeName
Słowenią placeName


In [11]:
# NER visualization
displacy.render(doc, style="ent")

In [12]:
# Additionally we've introduced a custom component called flexer

flexer = nlp.get_pipe("flexer")
# flexer allows you to obtain a new form with the desired morphosyntactic characteristics
# e.g. suppose you want to utilize the genitive case, in order to make user prompts more natural

prompt = "Autorem wiadomości jest Maciej"
print(prompt)
doc = nlp(prompt)

first_name = doc[3]
# you can simply pass the token to be inflected as the first, and the desired case as the second argument to the flexer.flex() method
print("'Maciej': nom -> gen")
inflected = flexer.flex(first_name, "gen") 
print(inflected)
print("\nMasz wiadomość od {}!\n".format(inflected))
# the feature names are in accordance with the NKJP tagset, for more information see here: http://nkjp.pl/poliqarp/help/ense2.html

# if you want to change more than one grammatical feature, pass them all in one string, separated by colons

txt = "Jan ugotował obiad."
print(txt)
ugotował = nlp(txt)[1]

print("\n'ugotował': m:sg -> f:pl\n")
print(flexer.flex(ugotował, "f:pl"))

# the order of individual features should not make a difference
print("\n'ugotował': m:sg -> pl:f\n")
print(flexer.flex(ugotował, "pl:f"))


# Because the flexer relies heavily on the tagger, flexing tokens taken from documents, will usually be better than
# inflecting individual words passed on as strings.
#print(flexer.flex("ugotował", "pl:f"))

# The range of possible alterations depends on the organization of the Morfeusz dictionary.
# some attributes may be unalterable at the moment (e.g. you cannot obtain "imperf" from "ugotował")
# but sometimes we can even obtain a different POS
print("\n'ugotował': fin -> ppas:neg\n")
print(flexer.flex(ugotował, "ppas:neg"))


Autorem wiadomości jest Maciej
'Maciej': nom -> gen
Macieja

Masz wiadomość od Macieja!

Jan ugotował obiad.

'ugotował': m:sg -> f:pl

ugotowały

'ugotował': m:sg -> pl:f

ugotowały

'ugotował': fin -> ppas:neg

nieugotowanego


In [13]:
# Simple token attributes
for token in doc:
    # Check if the token resembles a number
    if token.like_num:
        print(token)